In [1]:
#this notebook shows an example of the cell state classifier, that was defined on wildtype and control cells
#this following is a command used in Jupyter notebooks to display Matplotlib plots inline within the notebook
%matplotlib inline 

In [2]:
import sys #module that provides access to some variables used or maintaied by the interpreter
import numpy as np #import the numpy module and give it the alias np
import pandas as pd #import the pandas module and give it the alias pd
import seaborn as sns #import the seaborn module and give it the alias sns
import statsmodels 
import sklearn 
import matplotlib.pyplot as plt #import the pyplot module (from matplotlib) and give it the alias plt. The pyplot sub-module provides a convenient interface for creating plots and charts

sys.path.append('/home/jmalec/repo/coin_seq/MIMOSCA-2') #inside the single quotes add the path to the 'mimosca.py' file' an easy way to get the path is to go into the folder with the file and used the 'pwd' command in the terminal
import mimosca as mimosca

In [4]:
#the following line of code reads in a gene expression matrix in 10x format from the directory inside ' ' 
#DGE1 is a pandas.DataFrame object containing the gene expression data
#the read_10x() function is commonly used to read gene expression data from 10x Genomics output files
#IMPORTANT NOTE: when the repo is first cloned from GitHub the mm10 folder is a zip folder and needs to be unzipped 
DGE1=mimosca.read_10x('/home/jmalec/repo/coin_seq/MIMOSCA-2/Cell_States/mm10/')

#the following line of code collapses the 'DGE1' matrix to gene-level counts using the collapse2Gene method found in the mimosca module 
#DGE_genes is also a pandas.DataFrame but now instead of the original transcript-level counts, it has gene-level counts
#transcript-level counts are often measured using RNA sequencing, and count the abundance of individual RNA transcripts
#gene level counts represent the total abundance of all transcripts produced by a particular gene
DGE_genes=mimosca.collapse2gene(DGE1)

In [5]:
#the head method is from the Pandas package to show the first (default 5) rows of data from a DataFrame
#the rows represent genes from the mouse genome
DGE_genes.head()

,AAACATTGACTACG,AAACGCACCGTTAG,AAACGCTGACGGTT,AAACGCTGTCTCGC,AAACGGCTCGAATC,AAAGATCTACCCAA,AAAGCCTGAACCTG,AAAGCCTGCTCTTA,AAAGCCTGTCACGA,AAAGTTTGTTCTAC,...,TTGTCATGTCTCGC,TTTAGCTGAGCAAA,TTTAGCTGCATTCT,TTTCACGACACCAA,TTTCACGACTCTCG,TTTCAGTGAGCAAA,TTTCAGTGGCGATT,TTTCAGTGTGCAGT,TTTCCAGAATGGTC,TTTCTACTTGACCA
0610007P14Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0610009B22Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0610009L18Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0610009O20Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0610010F05Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#select variable genes
#the following line of code defines which gene names should be highlighted in the plot. This allows for a focus on specific genes of interest
highlight=['Tnf','Il6','Rsad2','Cxcl10']

#the tp10k_transform method (from the mimosca.py file) is called on the gene level count data frame from above
#mathematical transformation on gene expression data is applied to improve its suitability for downstream analysis. In this case the TP10K transformation method is used
#this normalization technique accounts for differences in gene length between samples 
#after this, the log2-transformed mean expression value for each gene in the DGE_genes data frame is calculated 
#the purpose of this is to obtain a measure of the overall expression level of each gene, as well as make the expression values more symmetrically distributed 
#the fano factor (which is a statistical measure of the variability of gene expression counts) is calculated 
#this input_mean parameter specifies the mean expression values calculated previously
#the resthresh and meanthresh parameters set the significant thresholds for the Fano factor and mean expression
#the highlight_genes parameter is used to display the genes specified in the previous list
#the plot parameter is used to display the plot on the screen
#in the plot that is generated, the x-axis represents the log2-transformed mean expression value of each gene, and the y-axis represents the fano factor
#the fano factor is calculated as the variance of the gene expression counts divided by the mean expression count
#the blue points represent genes that pass the significance threshold for mean expression level set by 'meanthresh' and these genes are considered to have high expression levels that are likely to be biologically relevant 
#the red points represent genes that pass the significance threshold for residual variance set by 'resthresh' and these genes are considered to have high variability in expression levels 
var_genes=mimosca.fano_variable(mimosca.tp10k_transform(DGE_genes),input_mean=np.log2(DGE_genes.mean(axis=1)+1),resthresh=0.02,meanthresh=0.5,highlight_genes=highlight,plot=1)

#these_genes is used to store the names of genes with significant residual variance or mean expression values 
these_genes=var_genes[np.logical_or(var_genes['residuals']>0.01,var_genes['mean']>2)].index

In [7]:
#Subset to variable genes
#the selected these_genes rows from the orignal dataframe are assigned to the new variable 'DGE_wt'
#this creates a new gene expression dataset
DGE_wt = DGE_genes.loc[these_genes]

In [8]:
#Z normalize Genes
#this line of code is used to standardize the preiviously selected data
#the tp10_transform method from the mimosca.py file is called again
#aftet this, the expression values are log2-transformed and normalized, and are passed to the Zgenes_floor() function in the mimosca.py file
#this function calculates the z-scores for each gene, which indicates how many strandard deviations that gene's expression value deviates from the mean expression value across all genes
#the floor paramater means any of the z-scores with a value lower than the value specified for floor are set to that value 
#the purpose of this is to help reduce the impact of outliers 
#the expression values are now standardized and stored in the new DGEwtZ DataFrame
DGEwtZ = mimosca.Zgenes_floor(np.log2(mimosca.tp10k_transform(DGE_wt)+1),floor=0.1)#,batchvec=batchwt)


In [17]:
#Facebook's Fast PCA (useful if using more than 10,000 cells) for calculating SVD, top 50 components
#the following line of code uses the mimosca module to perfrom PCA on the gene expression dataset DGEwtZ
#fbpca is used for 'dimensionality reduction and identification of underlying patterns in high-dimensional datasets"
#when working with gene expression data, fbpca is used to identify (in this case 50) principle components
#the principle components are supposed to capture the major sources of variation in the data
#ufb, sfb, vfb are matrices and represent the sigular value decompostion (SVD) of the input matrix (basically just a way to represent the matrix in a reduced-dimensionality form)

# print(pd.__version__) pretty sure we need a version earlier than 0.20.3
#[Ufb,Sfb,Vfb]=mimosca.fb_pca(DGEwtZ,k=50) #i think the pca method expects the input as a numpy array or matrix 
#M = np.asmatrix(DGEwtZ.values)
#[Ufb,Sfb,Vfb]=mimosca.fb_pca(M,k=50)
print(type(DGEwtZ))
[Ufb,Sfb,Vfb]=mimosca.fb_pca(DGEwtZ,k=50)

#[Ufb,Sfb,Vfb]=mimosca.fb_pca(DGEwtZ.values,k=50)


<class 'pandas.core.frame.DataFrame'>


AttributeError: 'Series' object has no attribute 'reshape'

In [ ]:
#Genes detected per cell
#cqc represents the number of non-zero elements in the DGE_wt matrix
#'DGE_wt>0' creates a boolean matrix, true corresponds to values > 0, and then sum() is used to sum the boolean values in the matrix
cqc=(DGE_wt>0).sum()

In [ ]:
#Calculate correlation of PC scores with genes detected
#correlation between the principal compoent scores and number of genes detected in the input gene expression data is calculated here
#the qc_cor list stores the correlation coefficients between each PC scores and the number of genes detected 
#the vfb matrix represents right singular vectors from the output of the fbpca performed earlier 
#the plot allows us to visualize how well each PC score correlates with the number of genes detected in the input gene expression data
#each principal component (PC) in pca is a linear combination of the original gene expression values for all genes in the dataset
#each PC is a weighted sum of the expression values of all the genes
#each PC represents a different pattern of gene expression across the samples, and the first view captures the largest variance in the data
#when performing pca we have the input matrix x which usually has each row corresponding to a gene, and each column representing a sample
#this matrix is used to calculate a covariance matrix, whos elements represent the covariance between pairs of the expression matrix
#the np.corrcoer() function is used to calculate the correlation coefficient between the PC score and the cqc variable, and they are added to the qc_cor list
qc_cor=[]
for pc in Vfb.columns:
    qc_cor.append(np.corrcoef(Vfb[pc],cqc)[0][1])
plt.plot(qc_cor)
plt.xlabel('PCs')
plt.ylabel('Correlation with Genes Detected')

## The Permutation and Noise Envelope Calculations Below Give The User Some Sense of the Number of PCs that are "significant"

In [12]:
# Permute gene expression matrix and calculate SVD
#the following line of code is using the permute_matrix in mimsoca.py to permute the gene expression matrix stored in DGE_wt
DGE_perm=mimosca.permute_matrix(DGE_wt)

#the tp10k_transform method to apply a transformation, followed by normalization
DGEwtZperm = mimosca.Zgenes(np.log2(mimosca.tp10k_transform(DGE_perm)+1))

#on the normalized matrix, the fb_pca function is called from the mimosca.py file to return 3 matrices
#these matrices represent the reduced-dimensionality form of the matrix
[Ufbperm,Sfbperm,Vfbperm]=mimosca.fb_pca(DGEwtZperm,k=50)

#plots the singular values ('sfb' and 'sfbperm')
#using the singular value decompostion of the matrix, sfb and sfbperm are calculated forthe actual gene expression matrix and the permuted gene expression matrix, respectivley
#we want to plot them against eachother to compare the two sets of values
#the purpose of doing this is because when we permuate the matrix, we obtain a set of singular values that represent the distribution of singular values we would expect to observe by chance
#this means if the singular values of the original matrix are similar, it suggests the observed structure of the original matrix is likely because of random variation 
plt.plot(Sfb,label='Actual Data')
plt.plot(Sfbperm,label='Permuted Data')
plt.legend()
plt.ylabel('Singular Values')
plt.xlabel('Ranked Eigenvectors')

AttributeError: 'DataFrame' object has no attribute 'ix'

In [ ]:
#calculate sensitivity of various PCs to noise and show noise envelope profile
#the noise envelope profile plots the correlation between the original eigenvector (computed during pca)
#and a range of eigenvectors generated by adding different levels of noise to the data (introducing random variations in the original data values)
#this helps evaluate how robust a method is to noise 
PC_cor=mimosca.PC_noise(DGEwtZ)

for i in range(20):
    plt.plot(np.linspace(-2,2,20),PC_cor.iloc[i],label=i)
plt.legend()
plt.xlabel('Log10(Added Noise)')
plt.ylabel('Correlation with Original Eigenvector')

In [ ]:
#calling the jackstraw function in the mimsoca.py file to perform jackstraw resampling on the standardized gene expression matrix DGEwtZ
#the function permutes each column of DGEwtZ and performs PCA on the resulting permuted matrix
#the resulting PCA scores are used to estimate the null distribution of each principal component
#the p-values of each principal component are calculated based on the portion of permuted PCA scores that are greater than the observed PCA score
#the point of finding p-values is to identify genes that are significantly associated with each PC
#a low p-value for a gene in a principal component means that gene is likely to be driving the variation in that PC
#this can provide insights into the biologcal pathways that underlie the observed variation in gene expression
PVALS=mimosca.jackstraw(DGEwtZ,sig_pcs=10,reps=1000,per=0.005)

In [ ]:
#Example of Jackstraw p-value calculation
#this code is showing the results of one of the Jackstraw analysis for one of the principal components
#the x-axis represents the loadings for a principal component which can be thought of as the contribution of each gene to the variation captured by that component
#the y-axis represents the signed Log10(q-value) obtained from the analysis, which tests the significance of each gene's contribution to the PC 
plt.scatter(Ufb[0],PVALS[0]) #changing the index from 0 to something else will repeat this for another PC
plt.axhline(1.3,c='red')
plt.axhline(-1.3,c='red')
plt.ylabel('Signed Log10(q-value) Jackstraw')
plt.xlabel('PCA loadings')
plt.title('PC2') #label error? should this be PC1? 

In [ ]:
#PCA is performed on tge DGEwtZ gene expression matrix 
#thresh specifies the threshold for the gene loadings, which is the contribution of each gene to the principle component
#gene loadings above the threshold will be considered important for the analysis 
#repin specifes the number of iterations for the random matrix resampling step that is used to estimate the distribution of the loadings
#df_pc is a PCRes object, one of the attributes being the 'pc' attribute, which is a matrix containing the principal components
df_pc = mimosca.PCA2GO(DGEwtZ,thresh=1.3,repin=1000,species='mouse')

In [ ]:
#creating a hierarchical clustering heatmap using the seaborn library with a specificed size and rotated y-axis labels
cg=sns.clustermap(df_pc,figsize=(10,20))
a=plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(),rotation=0)

In [ ]:
# Select 9 PCs based on the qc plots above
Vfb_cluster=Vfb.ix[:,range(9)] #is using the .ix indexing method (which is now deprecated and replaced by .loc or .iloc in newer versions of pandas)

In [ ]:
model = sklearn.manifold.TSNE(n_components=2, perplexity=20,verbose=2,init='pca',n_iter_without_progress=10000,min_grad_norm=0)
T_sne=model.fit_transform(Vfb_cluster)

In [ ]:
#Tsne plot colored by genes detected
T_sne=pd.DataFrame(T_sne)
plt.scatter(T_sne[0],T_sne[1],alpha=0.75,c=cqc,cmap='bwr')

In [ ]:
# You can use infomap if you have the libraries installed, this is probably slightly overclustered
info_labels=['cluster-'+str(x) for x in sklearn.cluster.k_means(Vfb_cluster,7)[1]]

In [ ]:
info_labels=np.array(info_labels)
unique_labs=np.unique(info_labels)
len(unique_labs)

In [ ]:
Tsneplot=T_sne.copy()
Tsneplot.columns=['1','2']
Tsneplot['clustering']=np.array(info_labels)
sns.lmplot('1','2',data=Tsneplot,hue='clustering',fit_reg=False)

In [ ]:
DGE_wtall=DGE1

In [ ]:
DGE_Z_expressed=mimosca.Zgenes(np.log2(mimosca.tp10k_transform(DGE_wtall[DGE_wtall.sum(axis=1)>20])+1))

In [ ]:
#calculate pairwsie differential expression between each cluster and if below some threshold merge
df_DE=mimosca.cluster_merger(DGE_Z_expressed,Vfb_cluster.index,info_labels)
df_DE

In [ ]:
#differential expression per cluster, each cluster against all others
df_minp = pd.DataFrame()
allcells=set(Vfb_cluster.index)
df_DC_cluster=pd.DataFrame()

for lab1 in unique_labs:
    cells1=Vfb_cluster.index[info_labels==lab1]
    cells2=list(allcells-set(cells1))
    print(lab1,len(cells1))

    fc,sl10pval=mimosca.ttest_DGE(DGE_Z_expressed[cells1],DGE_Z_expressed[cells2])
    #sl10pval=np.array(pd.DataFrame(sl10pval))
    df_minp[lab1]=np.sign(sl10pval)*(-np.log10(statsmodels.sandbox.stats.multicomp.fdrcorrection0(np.power(10,-np.abs(sl10pval)))[1]))#sl10pval 

In [ ]:
#convert infinity values to finite numbers
df_minp.index=DGE_Z_expressed.index
df_minp[df_minp>(1000.0)]=1000.0
df_minp[df_minp<(-1000.0)]=-1000.0

In [ ]:
df_minp.index=mimosca.genenames_from10x(df_minp.index)

In [ ]:
df_DEGO=mimosca.DE2GO(df_minp,df_minp.index,sig_thresh=1.3,species='mouse')

In [ ]:
#plot GO enrichments (remove .sample(100) and asjust figsize if you want to see them all)
cg=sns.clustermap(df_DEGO.sample(100),figsize=(20,20),metric='correlation')
a=plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(),rotation=0)

In [ ]:
Vfb_cluster_reproject=mimosca.project_ontoPC(Ufb.ix[:,range(9)],DGEwtZ)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,n_jobs=-1,oob_score=True,class_weight='balanced')
clf = clf.fit(Vfb_cluster_reproject,info_labels)
OOB=pd.DataFrame(clf.oob_decision_function_)

In [ ]:
#quantify how many cells are confidently classified
np.mean(OOB.max(axis=1)>0.5)

In [ ]:
#plot AUC curves for class membership

clusterun=np.unique(info_labels)

for i in range(np.shape(OOB)[1]):
    [fpr,tpr,thresholds]=sklearn.metrics.roc_curve(info_labels==clusterun[i],OOB[i])
    plt.plot(fpr,tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

In [ ]:
#Pseudo code for classifying new dataset

#DGEZ=New dataset you want to classify
#Vfb_new=mimosca.project_ontoPC(Ufb,DGEZ)
#PROBS=pd.DataFrame(clf.predict_proba(Vfb_new.ix[:,range(9)]))
#np.mean(PROBS.max(axis=1)>0.5)